In [1]:
import numpy as np
import pandas as pd

In [3]:
s = pd.Series([1, 3, 5, np.nan, 6, 8])
s

0    1.0
1    3.0
2    5.0
3    NaN
4    6.0
5    8.0
dtype: float64

In [6]:
!ls yelp-csv

yelp_academic_dataset_business.csv  yelp_academic_dataset_tip.csv
yelp_academic_dataset_review.csv    yelp_academic_dataset_user.csv


In [7]:
business = pd.read_csv('yelp-csv/yelp_academic_dataset_business.csv')  
#review = pd.read_csv('yelp-csv/yelp_academic_dataset_review.csv')  
#tip = pd.read_csv('yelp-csv/yelp_academic_dataset_tip_transposed.csv')  
user = pd.read_csv('yelp-csv/yelp_academic_dataset_user.csv')  


In [9]:
business.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,2818 E Camino Acequia Drive,{},1SWheh84yJXfytovILXOAQ,"Golf, Active Life",Phoenix,NaN,0,33.522143,-112.018481,Arizona Biltmore Golf Club,85016,5,3.0,AZ
1,30 Eglinton Avenue W,"{'GoodForMeal': ""{'dessert': False, 'latenight...",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5,ON
2,"1210 8th Street SW, Unit 220","{'BusinessParking': ""{'garage': False, 'street...",fcXOEZdXYeZqnQ3lGlOXmg,"Local Services, Professional Services, Compute...",Calgary,"{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ...",1,51.041771,-114.081109,Nucleus Information Service,T2R 1L3,5,2.0,AB
3,"6955 N Durango Dr, Ste 1116","{'BusinessParking': ""{'garage': False, 'street...",f2ZWZPENViL92BrFsIgR6w,"Beauty & Spas, Hair Salons",Las Vegas,"{'Monday': '8:30-20:0', 'Tuesday': '8:30-20:0'...",1,36.287312,-115.289540,Amazing Cuts,89149,116,4.5,NV
4,4 E University Dr,{},6KgGE8B1RsR7jc9R5nuH0Q,"American (Traditional), Restaurants",Tempe,NaN,0,33.422192,-111.939615,Ruby Tuesday,85281,9,2.5,AZ


In [4]:
business.count()

address         184927
attributes      163773
business_id     192609
categories      192127
city            192608
hours           147779
is_open         192609
latitude        192609
longitude       192609
name            192609
postal_code     191950
review_count    192609
stars           192609
state           192609
dtype: int64